### 概要

文章生成アルゴリズムの中で一般的なものの１つがマルコフ連鎖です。

データセットとして以下のような２つの文章があるとします。

・私　は　Axross　で　機械学習　の　勉強　を　します。     
・君　は　機械学習　の　エンジニア　を　目指して　います。

２つの文章には、助詞としての「は」と「の」と「を」が含まれるという共通点があります。それぞれの共通の助詞の地点でその先の単語をもう片方の文章のものと組み替えていくと、以下のような新規の文章を生成することができます。

・私は機械学習の勉強をします     
・君はAxrossで機械学習のエンジニアを目指しています

例の２つの文章から生成される文章は数パターンですが、データセットとして与える文章量を増やすことで生成される文章はより複雑なものになり、そのパターン数も増えていきます。これがマルコフ連鎖の仕組みになります。

このようにマルコフ連鎖は各要素から各要素への遷移過程をモデル化することができるので、Google検索のアルゴリズムであるPageRankにも使われています。リンクとWebページのネットワークという関係にこれを適応することで、ユーザーがWeb上でどのようにページ遷移するのかをモデル化することもできます。

今回のレシピではマルコフ連鎖で文章生成をするPythonライブラリであるMarkovifyを使って、夏目漱石の「坊ちゃん」をデータセットとして新作の文章を生成するプログラムを解説します。

In [19]:
from janome.tokenizer import Tokenizer
import markovify

In [31]:
def text_cleansing(text):
    # 改行、スペース、問題を起こす文字の置換
    table = str.maketrans({
        '。': '.',
        '\n': '',
        '\r': '',
        '…': '',
        '、': '',
        '々': '',
        '「': '',
        '」': '.',
        '(': '（',
        ')': '）',
        '[': '［',
        ']': '］',
        '"': '”',
        "'": "’",
    })
    text = text.translate(table)
    t = Tokenizer()
    # 分かち書き
    result = t.tokenize(text, wakati=True)
    result = list(result)
    # 1単語毎に間に半角スペース、文末には改行を挿入
    splitted_text = ""
    for i in range(len(result)):
        splitted_text += result[i]
        if result[i] != '。' and result[i] != '！' and result[i] != '？':
            splitted_text += ' '
        else:
            splitted_text += '\n'
    return splitted_text

In [32]:
with open('./data/bocchan.txt', mode='r', encoding='shift_jis') as f:
    text = f.read()
    
# テキストを単語毎に分割して記号を除去
splitted_text = text_cleansing(text)

マルコフ連鎖とは、ある単語の後にどのような単語が続くかを直前の単語をもとに決定することです。

１階マルコフ連鎖では、直前の１単語をキーとして次の単語をデータセットから探索して決めます。      
今回使用するのは2階マルコフ連鎖と言って、このキーが直前の２単語となっており上記と比較して単語を決定する条件が絞られるため、より意味のある文章を生成できるモデルとなります。

state_size は、マルコフ連鎖の階数になります。     
ここでは markovify.NewlineText は、状態サイズ：'2'（＝2階マルコフ連鎖）としてモデルを生成します。     
いくつ前の単語までをもととするか、その数（＝階層）を増減させることにより、探索する単語を制御することができます。

make_sentence メソッドは通常1回の呼び出しあたり最大10回まで、元のテキストと重複しないような文章の作成を試みます。成功した場合は、このメソッドは文を文字列として返します。成功しなかった場合は、None を返します。None が返る可能性があるために、成功するまで（文章が生成されて返るまで）while文でループさせています。

In [33]:
# データセットの量が不足していると確率で None が返却されるため、生成結果が変えるまでループさせる
sentence = None

while sentence == None:
    # モデル生成
    text_model = markovify.NewlineText(splitted_text, state_size=2)
    # モデルから文章を生成
    sentence = text_model.make_sentence(tries=100)

print(sentence.replace(' ', ''))

――面白いいかさま面白い.――さあ飲みたまえ.いかさま師をうんと云うほど酔わしてある奴はなんこを攫《つか》えるからな.それが赤シャツに挨拶をしたがそれから？
